# graph_mate

## Demo

***

First, we want to prepare some logging, so that we can see the output of what's going on.

In [1]:
import logging

logging.basicConfig(format="%(message)s")
logging.getLogger().setLevel(logging.NOTSET)

Next, we import the graph_mate library as well as numpy and pandas.

In [2]:
import graph_mate as gm
import numpy as np
import pandas as pd

Next we import the graph from the Graph500 format.

I have downloaded the graph and made it available locally.
If you want to reproduce it, you would have to download it from the [LDBC Graphalytics site](https://ldbcouncil.org/benchmarks/graphalytics/#data-sets).

We can now create a graph via `graph_mate` by loading from the local file.
We also pass in `gm.Layout.Deduplicated` as additional parameter to tell `graph_mate` to create a sorted adjacency list and deduplicate parallel edges.

In [3]:
g = gm.DiGraph.load('scale24.graph', layout = gm.Layout.Deduplicated)

Read 268435456 edges in 0.18s (17258.43 MB/s)
Creating directed graph
Computed degrees in 1.257856875s
Computed prefix sum in 12.315625ms
Computed target array in 2.315748708s
Finalized offset array in 5.90425ms
Sorted and deduplicated targets in 700.212083ms
Created outgoing csr in 4.322286333s.
Computed degrees in 1.050723958s
Computed prefix sum in 11.21ms
Computed target array in 1.526347875s
Finalized offset array in 4.804791ms
Sorted and deduplicated targets in 643.963208ms
Created incoming csr in 3.254451416s.
Created directed graph (node_count = 16777216, edge_count = 260379384)


Now we can run PageRank on the graph.

In [4]:
pr_result  = g.page_rank()

Finished iteration 0 with an error of 0.946463 in 159.495625ms
Finished iteration 1 with an error of 0.114661 in 157.28725ms
Finished iteration 2 with an error of 0.048918 in 156.766125ms
Finished iteration 3 with an error of 0.020391 in 156.670958ms
Finished iteration 4 with an error of 0.006081 in 157.068166ms
Finished iteration 5 with an error of 0.001342 in 156.872375ms
Finished iteration 6 with an error of 0.000232 in 156.805625ms
Finished iteration 7 with an error of 0.000033 in 156.3265ms


The result is a class that contains some properties about the execution of PageRank.

In [5]:
print(f"PageRank ran iterations: {pr_result.ran_iterations}")
print(f"PageRank computation took: {pr_result.micros / 1000.0} ms")

PageRank ran iterations: 8
PageRank computation took: 1282.766 ms


The `PageRankResult` has a `scores` method that returns the PageRank scores for all nodes.
The scores are returned as a numpy array without making a copy of the data.

In [6]:
pr_result.scores()

array([1.0839086e-08, 8.9587342e-09, 8.9406953e-09, ..., 8.9406953e-09,
       8.9746024e-09, 3.9346979e-08], dtype=float32)

Let's convert the scores to a pandas DataFrame.

In [7]:
scores = pd.DataFrame(pr_result.scores(), columns=['page_rank'])
scores

,page_rank
0,1.083909e-08
1,8.958734e-09
2,8.940695e-09
3,9.364135e-09
4,8.940695e-09
...,...
16777211,8.950504e-09
16777212,8.940695e-09
16777213,8.940695e-09
16777214,8.974602e-09


We can now calculate some statistics on the result

In [8]:
print(f"size = {scores.size:,}")
print(f"min = {scores.min()['page_rank']}")
print(f"max = {scores.max()['page_rank']}")
print(f"mean = {scores.mean()['page_rank']}")
print(f"median = {scores.median()['page_rank']}")

size = 16,777,216
min = 8.940695295223122e-09
max = 6.613731238758191e-05
mean = 1.5777033013364417e-08
median = 8.940695295223122e-09


Now we want to run WCC on the graph.

In [9]:
wcc_result = g.wcc()
print(f"WCC computation took: {wcc_result.micros / 1000.0} ms")

Afforest creation took 839.041µs
Link subgraph took 83.779875ms
Sample compress took 25.662541ms
Largest intermediate component 0 containing approx. 32% of the graph.
Get component took 323.25µs
Link remaining took 43.314333ms
Final compress took 6.803541ms


WCC computation took: 162.704 ms


Similar to the `PageRankResult`, we can get the component IDs for every nodes as a numpy array by calling the `components` method.

In [10]:
components = wcc_result.components()
components = pd.DataFrame(components, columns=['component'])


print(f"size = {components.size:,}")
unique_components = components.drop_duplicates()
print(f"component count = {unique_components.size:,}")

size = 16,777,216
component count = 7,909,210


Now we want to count the total number of triangles in the graph.

We have to convert the graph to an undirected graph first.

In [11]:
ug = g.to_undirected()

Creating undirected graph
Computed degrees in 1.208045583s
Computed prefix sum in 10.864166ms
Computed target array in 1.638902916s
Finalized offset array in 2.873458ms
Created csr in 2.862828791s.
Created undirected graph (node_count = 16777216, edge_count = 260379384)


If we are pressed for memory we can delete the directed graph.
The undirected graph is not a view but a full copy of the graph.

In [12]:
del g

Counting triangles benefits from an adjacency list that is sorted by degree.
We can sort the adjacency list by calling the `reorder_by_degree` method.

Note that `reorder_by_degree` modifies the graph in place can only run if there are no references to neighbors lists of any node.

In [13]:
neighbors = ug.neighbors(0)

try:
    ug.reorder_by_degree()
except ValueError as e:
    print(e)

del neighbors

Graph cannot be reordered because there are references to this graph from neighbor lists.


In [14]:
ug.reorder_by_degree()

Relabel: sorted degree-node-pairs in 142.235083ms
Relabel: built degrees and id map in 18.187791ms
Relabel: built and sorted targets in 1.527368208s


Now we can count the number of global triangles in the graph.

In [15]:
tc = ug.global_triangle_count()

print(f"TC: found {tc.triangles} triangles in {tc.micros / 1000.0 / 1000.0} seconds")

Computed 10,276,536,795 triangles in 60.788832666s


TC: found 10276536795 triangles in 60.789366 seconds
